This notebook serves as a playground for experimenting with data science using Python and Spotify APIs. This one focuses on the a user logging in and extracting a user's data

First we install a few necessary libraries

In [1]:
!pip install -q numpy
!pip install -q pandas
!pip install requests

Next, we are using a spotify API client made using a tutorial found here: [video](https://www.youtube.com/watch?v=xdq6Gz33khQ) [GitHub](https://github.com/codingforentrepreneurs/30-Days-of-Python/blob/master/tutorial-reference/Day%2019/notebooks/spotify_client.py)

This class has been altered to add more access functions, and in this notebook, added the ability for a user to login.

In [2]:
import base64
import datetime
from urllib.parse import urlencode

import requests

#Follows Spotify's "Implicit Grant" Authorization flow
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    authorization_url = "https://accounts.spotify.com/authorize"
    
    #external functions ("public")
    

    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')

    def get_playlist(self, _id):
        return self.get_resource(_id, resource_type='playlists')

    def get_playlist_tracks(self, _id):
        return self.get_resource(_id, resource_type='playlists', addendum='tracks')

    def get_track_features(self, _id):
        return self.get_resource(_id, resource_type='audio-features')

    def get_track_analysis(self, _id):
      return self.get_resource(_id, resource_type='audio-analysis') 

    def search(self, query=None, operator=None, operator_query=None, search_type='artist', limit=20 ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower(), "limit": str(limit)})
        return self.base_search(query_params)


    #internal functions ("private")


    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1', addendum=''):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}/{addendum}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()

    #new functions for user functions

    def get_current_user(self):
        endpoint = f"https://api.spotify.com/v1/me"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

    def get_current_user_recently_played(self):
        endpoint = f"https://api.spotify.com/v1/me/player/recently-played"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

    def get_current_user_top(self, top_type='tracks', time_range='medium_term'):
        #endpoint = f"https://api.spotify.com/v1/me/top/{top_type}?limit=50&time_range{time_range}"
        endpoint = f"https://api.spotify.com/v1/me/top/{top_type}?time_range={time_range}&limit=50"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            print(r.status_code)
            return {}
        return r.json()

    def get_current_user_top_amount(self, top_type='tracks', time_range='medium_term', amount=50):
        headers = self.get_resource_header()
        received_resources = 0
        return_list = []
        limit = amount % 50
        while received_resources < amount:
          endpoint = f"https://api.spotify.com/v1/me/top/{top_type}?time_range={time_range}&limit={str(limit)}&offset={received_resources}"
          r = requests.get(endpoint, headers=headers)
          if r.status_code not in range(200, 299):
              return {}
          return_list.append(r.json()['items'])

    def get_user(self, user):
        endpoint = f"https://api.spotify.com/v1/users/{user}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()


Let's not forget to import the all powerful datascience libraries: NumPy and pandas !

In [3]:
import webbrowser
import json

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import math

Now, we pass our client ID and client secret from our Spotify Dev project into an instance of the spotify_client class. Paste in your own within the quotes. To get a your Spotify credentials, visit [here](https://developer.spotify.com/dashboard/login)

In [4]:
client_id = ""
client_secret = ""


Add the appropriate scopes for getting the correct access token for a user.

In [5]:
scopes = ['user-read-email', 'user-read-private', 'playlist-read-collaborative', 'playlist-modify-public', 'playlist-read-private', 'playlist-modify-private', 'user-library-read', 'user-top-read', 'user-read-recently-played', 'user-modify-playback-state', 'user-read-playback-state']
scopes_string = ' '.join(scopes)
scopes_string


'user-read-email user-read-private playlist-read-collaborative playlist-modify-public playlist-read-private playlist-modify-private user-library-read user-top-read user-read-recently-played user-modify-playback-state user-read-playback-state'

Creating the GET request to log a user in. Follow the link generated to allow access to user data

In [6]:
authorization_url = "https://accounts.spotify.com/authorize"


params = {
            "client_id": client_id,
            "response_type": "token",
            "redirect_uri": "http://localhost:7777/callback",
            "scope": scopes_string
        } 
r = requests.get(authorization_url, params)
print("Successful if this is 200: " + str(r.status_code))
print("URL to paste in browser: " + str(r.url))

Successful if this is 200: 200
URL to paste in browser: https://accounts.spotify.com/login?continue=https%3A%2F%2Faccounts.spotify.com%2Fauthorize%3Fscope%3Duser-read-email%2Buser-read-private%2Bplaylist-read-collaborative%2Bplaylist-modify-public%2Bplaylist-read-private%2Bplaylist-modify-private%2Buser-library-read%2Buser-top-read%2Buser-read-recently-played%2Buser-modify-playback-state%2Buser-read-playback-state%26response_type%3Dtoken%26redirect_uri%3Dhttp%253A%252F%252Flocalhost%253A7777%252Fcallback%26client_id%3D5de0c3326eed4e17a4e2b226fc57a68f


Now paste the entire url you were sent to after allowing permissions between the quotes for "response ="

In [12]:
#paste the entire url you are sent to here as a string(in quotes
response = ''
response_list_raw = response.split('&')

response_list = []
#hard coded extracting token and expires
for item in response_list_raw:
  response_list.append(item.split('='))

response_list

access_token = response_list[0][1]
expires_in = response_list[2][1]
print(access_token)
print(expires_in)

BQBJhh2JbCu9D1_yVbq907jBzdAMP598S4pNlwlKnACSTDoPq_A2S_w-lR1UhDLd0_1PDzR6Bjr_5gvPndTTLLEhoblPYG_hUnqgdX-4UjavVxqGmcusY9qaL9mhYWPj4WxQJuaCuvIGbCOh24A3TKohUvPcFeAOWMChPazogDkDnGTaYB4SNQ2V4g537GddDWzcoMxgiABeUdd6yBRUE65_kFhdXh8ihYN-3yORlcoZXEz6mvtbqfjZrDJvTi6pvmA
3600


Now I have a token! Let's try it out by trying to access generic data

In [13]:
spotify_nouser = SpotifyAPI(client_id, client_secret)
spotify_user = SpotifyAPI(client_id, client_secret)







In [14]:
spotify_user.access_token = access_token
spotify_user.access_token_expires = datetime.datetime.now() + datetime.timedelta(seconds=3600)
spotify_user.access_token_did_expire = False

In [15]:
nu = spotify_nouser.search("Khalid", search_type='artist')

u = spotify_user.search("Khalid", search_type='artist')

print(spotify_nouser.access_token)
print(nu)
print(spotify_user.access_token)
print(u)

BQDS8qUtiBYv8x4nkyEnzfK9m-ZNsdSQNZcsDVqTCJeuHnmL6Aft-V5U3ujjcOXTL-iWnophK7aOzQSIdkk
{'artists': {'href': 'https://api.spotify.com/v1/search?query=Khalid&type=artist&offset=0&limit=20', 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6LuN9FCkKOj5PcnpouEgny'}, 'followers': {'href': None, 'total': 14112151}, 'genres': ['pop'], 'href': 'https://api.spotify.com/v1/artists/6LuN9FCkKOj5PcnpouEgny', 'id': '6LuN9FCkKOj5PcnpouEgny', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5eb012b37d6dec8872b18524f78', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174012b37d6dec8872b18524f78', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178012b37d6dec8872b18524f78', 'width': 160}], 'name': 'Khalid', 'popularity': 91, 'type': 'artist', 'uri': 'spotify:artist:6LuN9FCkKOj5PcnpouEgny'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/7w9NGVft7cOEEa3CGGILZM'}, 'followers': {'href': None

It works! Now let's try to access some user data.

In [16]:
current_user_raw = spotify_user.get_current_user()
current_user_raw

{'country': 'US',
 'display_name': 'kyle.sellers',
 'email': 'sellerskyle@live.com',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False},
 'external_urls': {'spotify': 'https://open.spotify.com/user/kyle.sellers'},
 'followers': {'href': None, 'total': 10},
 'href': 'https://api.spotify.com/v1/users/kyle.sellers',
 'id': 'kyle.sellers',
 'images': [{'height': None,
   'url': 'https://i.scdn.co/image/ab6775700000ee8521597027ae50724152aad9b4',
   'width': None}],
 'product': 'premium',
 'type': 'user',
 'uri': 'spotify:user:kyle.sellers'}

In [17]:
recently_played = spotify_user.get_current_user_recently_played()
recently_played

{'cursors': {'after': '1627056154006', 'before': '1627027780232'},
 'href': 'https://api.spotify.com/v1/me/player/recently-played',
 'items': [{'context': None,
   'played_at': '2021-07-23T16:02:34.006Z',
   'track': {'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6nZULdUsobKEqqiGgTDNda'},
       'href': 'https://api.spotify.com/v1/artists/6nZULdUsobKEqqiGgTDNda',
       'id': '6nZULdUsobKEqqiGgTDNda',
       'name': 'Narco Debut',
       'type': 'artist',
       'uri': 'spotify:artist:6nZULdUsobKEqqiGgTDNda'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO'

Great, that worked! Let's see what your top songs are!

In [18]:
top_tracks_raw = spotify_user.get_current_user_top(top_type='tracks', time_range='long_term')
top_tracks = top_tracks_raw['items']
len(top_tracks)

50

In [19]:
top_artists_raw = spotify_user.get_current_user_top(top_type='artists', time_range='short_term')
top_artists = top_artists_raw['items']
len(top_artists)

47

In [20]:
indices = {'resource_types':['tracks','artists'],'time_frames':['short_term','medium_term', 'long_term']}

raw_file_data = {}
for resource_type in indices['resource_types']:
  for time_frame in indices['time_frames']:
    index_string = resource_type + "_" + time_frame
    raw_file_data[index_string] = spotify_user.get_current_user_top(top_type=resource_type, time_range=time_frame)['items']

raw_file_data['profile'] = spotify_user.get_current_user()

In [23]:
file_name = raw_file_data['profile']['id'] + '.json'
with open(file_name, 'w') as outfile:
    json.dump(raw_file_data, outfile)


Let's put this data in a json file in a json file and download it!

In [22]:
!ls

kyle.sellers.json  sample_data


In [ ]:
# Download the file.
from google.colab import files
files.download(file_name)

Begin user Data Processing

In [25]:
with open(file_name) as json_file:
    user_data = json.load(json_file)


In [26]:
#adding ranking to each item in user data

for resource_type in indices['resource_types']:
  for time_frame in indices['time_frames']:
    rank = 0
    for resource in user_data[resource_type + '_' + time_frame]:
      rank += 1
      resource['rank'] = rank

user_data

{'artists_long_term': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/05MlomiA9La0OiNIAGqECk'},
   'followers': {'href': None, 'total': 49965},
   'genres': ['brighton indie',
    'british math rock',
    'progressive post-hardcore'],
   'href': 'https://api.spotify.com/v1/artists/05MlomiA9La0OiNIAGqECk',
   'id': '05MlomiA9La0OiNIAGqECk',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab6761610000e5ebef86feb3e2b9c57897ddc453',
     'width': 640},
    {'height': 320,
     'url': 'https://i.scdn.co/image/ab67616100005174ef86feb3e2b9c57897ddc453',
     'width': 320},
    {'height': 160,
     'url': 'https://i.scdn.co/image/ab6761610000f178ef86feb3e2b9c57897ddc453',
     'width': 160}],
   'name': 'Delta Sleep',
   'popularity': 50,
   'rank': 1,
   'type': 'artist',
   'uri': 'spotify:artist:05MlomiA9La0OiNIAGqECk'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/3mIj9lX2MWuHmhNCA7LSCW'},
   'followers': {'href': None, 'total': 500997

In [27]:



all_tracks_raw = []

all_tracks_raw = all_tracks_raw + user_data['tracks_short_term']
all_tracks_raw = all_tracks_raw + user_data['tracks_medium_term']
all_tracks_raw = all_tracks_raw + user_data['tracks_long_term']


all_artists_raw = []

all_artists_raw = all_artists_raw + user_data['artists_short_term']
all_artists_raw = all_artists_raw + user_data['artists_medium_term']
all_artists_raw = all_artists_raw + user_data['artists_long_term']


In [28]:
print(len(all_tracks_raw))

set_of_track_jsons = {json.dumps(d, sort_keys=True) for d in all_tracks_raw}
all_tracks_no_dup = [json.loads(t) for t in set_of_track_jsons]

print(len(all_tracks_no_dup))

print(len(all_artists_raw))

set_of_artist_jsons = {json.dumps(d, sort_keys=True) for d in all_artists_raw}
all_artists_no_dup = [json.loads(t) for t in set_of_artist_jsons]

print(len(all_artists_no_dup))

150
150
147
146


Artist visualization for user top tracks

In [29]:
#Finding avg popularity of artists you listen to
total_artist_pop = 0
for artist in all_artists_no_dup:
  total_artist_pop = total_artist_pop + artist['popularity']

avg_artist_pop = total_artist_pop / len(all_artists_no_dup)
avg_artist_pop

52.363013698630134

Here's a bar graph of your top genres! (Hint: right click the image and open in new tab to zoom in)

In [30]:
#Most prominent genre's of artists you listen to

genre_dict = {}


for artist in all_artists_no_dup:
  for genre in artist['genres']:
    if genre in genre_dict:
      genre_dict[genre] = genre_dict[genre] + 1
    else:
      genre_dict[genre] = 1

genre_dict
sort_genre_dict = sorted(genre_dict.items(), key=lambda x: x[1], reverse=True)
print("dimensions: " + str(len(sort_genre_dict)) + " x " + str(len(sort_genre_dict[0])))
df = pd.DataFrame(sort_genre_dict, columns=['genre', 'count'])
sns.set(style="darkgrid")


fig_dims = (900, 20)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x='genre', y='count', data=df, ax=ax)


dimensions: 157 x 2


In [31]:
#finding percent of your top songs that are by your top artists

top_artists_names = []

for artist in all_artists_no_dup:
  top_artists_names.append(artist['name'])

all_tracks_no_dup[0]

top_tracks_names_plus_artists = []
for track in all_tracks_no_dup:
  name = track['name']
  artist_names = []
  for artist in track['artists']:
    artist_names.append(artist['name'])
  top_tracks_names_plus_artists.append({'name': name, 'artists': artist_names})




  



In [32]:
top_tracks_names_plus_artists

num_tracks_by_fav_artists = 0

tracks_not_by_fav_artists = []



for track in top_tracks_names_plus_artists:
  this_track_by_fav_artist = False
  for artist in track['artists']:
    for top_artist in top_artists_names:
      if artist == top_artist:
        num_tracks_by_fav_artists = num_tracks_by_fav_artists + 1
        this_track_by_fav_artist = True
  if not this_track_by_fav_artist:
    tracks_not_by_fav_artists.append(track)

print(num_tracks_by_fav_artists)
print(tracks_not_by_fav_artists)
print(top_artists_names)

261
[{'name': 'Face My Fears', 'artists': ['Wind Walkers']}, {'name': 'Brand New Day', 'artists': ['BABYMETAL', 'Tim Henson', 'Scott LePage']}, {'name': 'Duet', 'artists': ['Itzamna']}, {'name': 'Change Your Mind', 'artists': ['NOVACUB']}, {'name': 'Still Posi', 'artists': ['Sunrise Skater Kids']}, {'name': 'The Dreamer', 'artists': ['White Moth Black Butterfly']}, {'name': 'God Damn', 'artists': ['All Get Out']}, {'name': 'Speed Bump', 'artists': ['Cheem']}, {'name': 'Dirty Talk', 'artists': ['Nicolas Rage']}, {'name': 'Hanging By A Thread', 'artists': ['PhaseOne', 'Micah Martin']}, {'name': 'Can I', 'artists': ['Tedy']}, {'name': 'Twinkle Lights', 'artists': ['The Sonder Bombs']}, {'name': 'Try', 'artists': ['Olly Steele', 'Ian Kenny']}, {'name': 'Violet', 'artists': ['patchymate', 'abbot']}, {'name': 'Abstract Dynamics', 'artists': ['Deer Belling Café']}, {'name': "It's Only Smiles", 'artists': ['Periphery']}, {'name': 'Amaranthine', 'artists': ['Sal3m']}, {'name': 'I Met You in the

In [34]:
#find all top artists that match a provided genre
target_genre = 'pop rap'

artists_belonging_to_genre = []

for artist in all_artists_no_dup:
  for genre in artist['genres']:
    if target_genre == genre:
      artists_belonging_to_genre.append(artist['name'])

artists_belonging_to_genre

['Logic', 'Jon Bellion', 'Lil Wayne', 'WILLOW', 'Lil Wayne']

The previous graph was a sum of all artists that fit in a genre, over your entire listening history on spotify. Let's weight these artists so the ones you listened to recently are more important

In [35]:
#weighted graph of favorite genres

#lets start small with just short term songs weighting

short_term_weight = 0.8
medium_term_weight = 0.1
long_term_weight = 0.1

short_term_scaling = 10
medium_term_scaling = 5
long_term_scaling = 3

time_frame_weights_scalings = {
    'short_term': {'weight': short_term_weight,'scaling':short_term_scaling},
    'medium_term': {'weight': medium_term_weight,'scaling':medium_term_scaling},
    'long_term': {'weight': long_term_weight,'scaling':long_term_scaling}
}

# test_list = np.arange(1,51,1)
# test_list

# scaling = 10
# amount_per_step = ((scaling-1)/scaling) / (len(test_list) - 1)
# amount_per_step

# scaled_list = []

# for item in test_list:
#   val = 1/scaling + ((len(test_list) - (item)) * amount_per_step)
#   scaled_list.append(val)

# scaled_list
genre_dict_scaled = {}

for time_frame in indices['time_frames']:
  scaling = time_frame_weights_scalings[time_frame]['scaling']
  for artist in user_data['artists_' + time_frame]:
    amount_per_step = ((scaling-1)/scaling) / (len(user_data['artists_' + time_frame]) - 1)
    for genre in artist['genres']:
      if genre in genre_dict_scaled:
        genre_dict_scaled[genre] += (1/scaling + ((len(user_data['artists_' + time_frame]) - (artist['rank'])) * amount_per_step)) * time_frame_weights_scalings[time_frame]['weight']
      else:
        genre_dict_scaled[genre] = (1/scaling + ((len(user_data['artists_' + time_frame]) - (artist['rank'])) * amount_per_step)) * time_frame_weights_scalings[time_frame]['weight']
      # print((1/scaling + ((len(user_data['artists_' + time_frame]) - (item)) * amount_per_step)) * time_frame_weights_scalings[time_frame]['weight'])


genre_dict_scaled
sorted_genre_dict_scaled = sorted(genre_dict_scaled.items(), key=lambda x: x[1], reverse=True)
sorted_genre_dict_scaled

df_scaled = pd.DataFrame(sorted_genre_dict_scaled, columns=['genre', 'weight'])
sns.set(style="darkgrid")


fig_dims = (900, 20)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x='genre', y='weight', data=df_scaled, ax=ax)



In [36]:
#log plot?
sorted_genre_dict_scaled_log = []


minimum = sorted_genre_dict_scaled[len(sorted_genre_dict_scaled) - 1][1]


for entry in sorted_genre_dict_scaled:
  new_entry = {}
  new_entry[entry[0]] = math.log(entry[1]/minimum)
  sorted_genre_dict_scaled_log.append(new_entry)

sorted_genre_dict_scaled_log

# df_scaled_log = pd.DataFrame(sorted_genre_dict_scaled_log, columns=['genre', 'weight'])
# sns.set(style="darkgrid")
# df_scaled_log

# fig_dims = (900, 20)
# fig, ax = plt.subplots(figsize=fig_dims)
# sns.barplot(x='genre', y='weight', data=df_scaled_log, ax=ax)

[{'progressive post-hardcore': 5.9752941351993805},
 {'instrumental math rock': 5.542539762998044},
 {'swancore': 5.531341549875872},
 {'british math rock': 4.863508428653536},
 {'instrumental rock': 4.820245786277613},
 {'pop': 4.796296405147196},
 {'math rock': 4.598371022226193},
 {'math pop': 4.533486411171483},
 {'midwest emo': 4.463368750293616},
 {'djent': 4.460798997885958},
 {'electropop': 4.249546786840071},
 {'pop punk': 4.198208839285987},
 {'dreamo': 4.168573300030217},
 {'progressive jazz fusion': 4.158240500431653},
 {'instrumental progressive metal': 4.127134385045092},
 {'metalcore': 4.108237549988472},
 {'instrumental djent': 4.033583445336916},
 {'edm': 3.9284970307123506},
 {'screamo': 3.8855350821427876},
 {'pixie': 3.859636319408721},
 {'sacramento indie': 3.859636319408721},
 {'dance pop': 3.7708246577670237},
 {'emo': 3.736006287291775},
 {'oxford indie': 3.736006287291775},
 {'australian metal': 3.724413311815813},
 {'new jersey indie': 3.690718932486858},
 {'p

In [38]:
analytics = df_scaled.describe()
type(analytics)
mean = analytics['weight']['mean']
sorted_genre_dict_scaled
gt_mean = [x for x in sorted_genre_dict_scaled if x[1] > mean]
gt_mean

[('progressive post-hardcore', 7.87167701863354),
 ('instrumental math rock', 5.106512866015972),
 ('swancore', 5.049648033126294),
 ('british math rock', 2.58955338657202),
 ('instrumental rock', 2.4799112688553686),
 ('pop', 2.4212244897959185),
 ('math rock', 1.9864477965099083),
 ('math pop', 1.861650399290151),
 ('midwest emo', 1.7355871044069804),
 ('djent', 1.7311328009464657),
 ('electropop', 1.4014729370008874),
 ('pop punk', 1.3313398402839398),
 ('dreamo', 1.292463768115942),
 ('progressive jazz fusion', 1.2791777580597459),
 ('instrumental progressive metal', 1.2400000000000002),
 ('metalcore', 1.2167879325643303),
 ('instrumental djent', 1.1292576160899144),
 ('edm', 1.0166104702750665),
 ('screamo', 0.9738598047914818),
 ('pixie', 0.9489618456078084),
 ('sacramento indie', 0.9489618456078084),
 ('dance pop', 0.8683170659568178),
 ('emo', 0.8386039633244602),
 ('oxford indie', 0.8386039633244602),
 ('australian metal', 0.82893818396924),
 ('new jersey indie', 0.80147293700